# **Using a SciPy sparse matrix to model the fire station problem**

***firestation_scipy.ipynb***

In this example, we solve the fire station location problem using a SciPy sparse matrix with the [xpress.Dot()](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/xpress.Dot.html) operator.

*A version of FICO&reg; Xpress >=9.5 is required for being able to use SciPy matrices in the xp.Dot() operator.*

&copy; Copyright 2025 Fair Isaac Corporation

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0.
 
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

This example uses FICO&reg; Xpress software. By running it, you agree to the Community License terms of the [Xpress Shrinkwrap License Agreement](https://community.fico.com/s/contentdocument/06980000002h0i5AAA) with respect to the FICO&reg; Xpress software. See the [licensing options](https://www.fico.com/en/fico-xpress-trial-and-licensing-options) overview for additional details and information about obtaining a paid license.

In [ ]:
# Install the xpress package
%pip install -q xpress

## Problem description and formulation

The fire station problem attemps to minimize the number of fire stations to build amongst a set of towns, with each town being a candidate for hosting a fire station. Each town must be served by a fire station built on a town with a travel time no longer than a pre-defined threshold (e.g. 15 minutes).

The travel times between six towns can be described as:

|   | Town 0 | Town 1 | Town 2 | Town 3 | Town 4 | Town 5 |
|---|---|---|---|---|---|---|
| **Town 0** | 0 | 15 | 25 | 35 | 35 | 25 |
| **Town 1** | 15 | 0 | 30 | 40 | 25 | 15 |
| **Town 2** | 25 | 30 | 0 | 20 | 30 | 25 |
| **Town 3** | 35 | 40 | 20 | 0 | 20 | 30 |
| **Town 4** | 35 | 25 | 35 | 20 | 0 | 19 |
| **Town 5** | 25 | 15 | 25 | 30 | 19 | 0 |

Therefore, a binary matrix indicating whether each town can serve another considering a travel time threshold of 15 minutes would result in the following:

|   | Town 0 | Town 1 | Town 2 | Town 3 | Town 4 | Town 5 |
|---|---|---|---|---|---|---|
| **Town 0** | 1 | 1 | 0 | 0 | 0 | 0 |
| **Town 1** | 1 | 1 | 0 | 0 | 0 | 1 |
| **Town 2** | 0 | 0 | 1 | 0 | 0 | 0 |
| **Town 3** | 0 | 0 | 0 | 1 | 0 | 0 |
| **Town 4** | 0 | 0 | 0 | 0 | 1 | 0 |
| **Town 5** | 0 | 1 | 0 | 0 | 0 | 1 |


Based on this data, we can define a mathematical formulation for the problem as follows:

Let $build_{i} \in \{0,1\}, \forall i \in \mathcal{T}$ represent the **decision of whether to select town $i \in \mathcal{T}$ for building a fire station**, and $avail_{i,j} \in \{0,1\}, \forall i,j \in \mathcal{T}$ a **given binary matrix representing whether town $i$ can serve town $j$** within the predefined travel time limit. The problem can be therefore formulated as follows:
$$
\min \sum_{i \in \mathcal{T}} build_{i}
$$

subject to:

* Each town $j$ must be served by at least one eligible fire station: 
$$
\sum_{i \in \mathcal{T}} avail_{i,j} \cdot build_{i} \geq 1, \quad \forall j \in \mathcal{T} \\
x_{i} \in \{0,1\}, \forall i \in \mathcal{T}
$$

## Data preparation

In such problem, the $A$ matrix can potentially be a large sparse matrix, in case e.g. most towns can only serve a few others or none at all.  

SciPy has a module [scipy.sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html) for handling 2D sparse data in a compact format, which allows representing 2D arrays by using the row/column indices of the non-zero values only. Sparse array formats allow building models more efficiently by avoiding iterating over all the elements (including the zeros) of a conventional array.

**The [xp.Dot()](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/xpress.Dot.html) operator supports the most common SciPy sparse matrix formats (CSR and CSC)**, allowing to compute the product of a 1-D NumPy array of variables or expressions with a sparse matrix of
numbers.

In the code cell below, an instance with $T$ = 6 towns is created and the travel times between towns is given as a NumPy array. Then, a time limit of 15 minutes is defined to compute the $A$ matrix of binary values named as <tt>avail</tt>. The 2D array is then converted into a SciPy sparse matrix format (using the method <tt>csr_array</tt>) before printing both matrices for visuzalization in the output log.

In [ ]:
import xpress as xp
import numpy as np
import scipy
import time

num_towns = 6     # Number of towns

t_time = np.array([[ 0.,15.,25.,35.,35.,25.],    # Travel times between towns
 [15., 0.,30.,40.,25.,15.],
 [25.,30., 0.,20.,30.,25.],
 [35.,40.,20., 0.,20.,30.],
 [35.,25.,35.,20., 0.,19.],
 [25.,15.,25.,30.,19., 0.]])

avail = (t_time <= 15).astype(int)                # NumPy array of binary values which are equal to 1 when t_time <= 15, otherwise 0

avail_sparse = scipy.sparse.csr_array(avail)      # Convert to SciPy sparse matrix format

print("NumPy format: ", avail, sep="\n")
print("SciPy format: ", avail_sparse, sep="\n")

## Model implementation and results

The optimization problem is constructed in the code cell below, starting with the creation of a problem instance before adding the set of binary variables $x$. By using [p.addVariables](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/problem.addVariables.html) with an integer argument, a NumPy array of variables is created.

Then, the [xp.Dot()](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/xpress.Dot.html) operator is used for applying the <tt>dot</tt> product between matrix $A$ and the array of variables $x$. The result is a set of $|T|$ constraints created with a right-hand side equal of "1", which is broadcasted to all constraints.

After setting the objetive to minimize the number of towns selected to build a fire station, the problem is optimized, exported in LP format, and the final solution and objective value are printed.

In [ ]:
p = xp.problem()      # Create Xpress problem

x = p.addVariables(num_towns, vartype=xp.binary)     # Creates NumPy array of variables when integers are passed

# Serve all towns, amongst those eligible to be selected for each
p.addConstraint(xp.Dot(avail_sparse,x) >= 1) # Creates T constraints with RHS = 1

p.setObjective(xp.Sum(x)) # Minimize number of towns selected for a fire station

p.optimize()

p.write("prob.lp")

print("Minimum number of stations: ", round(p.attributes.objval))
print("Located at towns",[s+1 for s in range(num_towns) if p.getSolution(x[s]) >= 0.99])

## Comparison between using NumPy and SciPy arrays in the *xp.Dot()* operator for a large scale instance


In this part, we use the *randint* method of the *random* NumPy module to generate travel times randomly between 0 and 100 minutes for each pair of $T$ towns (uniform distribution, matrix not symmetrical). The number of towns (T) can be modified by the user to scale the size of the instance up or down.

A threshold is then applied to define the density of the matrix. A value of 1 will correspond to (approximately) a matrix density of 1%, the percentage of non-zeros in the matrix.

The matrix is then printed in both NumPy and SciPy formats, such as in the previous part.

In [ ]:
num_towns = 10000         # Number of towns

rndseed = 10
np.random.seed(rndseed)

t_time = np.random.randint(0, 100, (num_towns, num_towns))

avail = np.array((t_time <= 1).astype(int))      # NumPy array of binary values which are equal to 1 when t_time <= 1, otherwise 0

avail_sparse = scipy.sparse.csr_array(avail)     # Convert to SciPy sparse matrix format CSR

print(avail)
print(avail_sparse)

Lastly, the *time* package is used to record the time needed to build the set of 10000 constraints using NumPy arrays versus using SciPy sparse arrays when applying the *xp.Dot()* product.

Although using NumPy arrays with the *xp.Dot()* operator already allows building constraints much more efficiently than using conventional Python lists (by performing loop operations in lower level C and thus introducing a much lower overhead), we can see that further gains in modelling performance can be achieved by using SciPy arrays for matrices that are highly sparse. 
 
By changing the threshold value in the previous code cell and running both code cells again, one can verify that the lower the density of the original matrix, the more gains are achieved when using SciPy versus using NumPy arrays with regards to model building time. 

In [8]:
p = xp.problem()    # Create Xpress problem

x = p.addVariables(num_towns, vartype=xp.binary)     # Creates NumPy array of variables when integers are passed

# Constraints created with a NumPy matrix
start = time.time() # records start time
p.addConstraint(xp.Dot(avail,x) >= 1)        # Creates T constraints with RHS = 1
stop = time.time() # record time
print(f"Time with NumPy coefs matrix: {(stop-start):.03f} secs.")

# Constraints created with a SciPy sparse matrix
p.addConstraint(xp.Dot(avail_sparse,x) >= 1) # Creates T constraints with RHS = 1
end = time.time() # record end time
print(f"Time with SciPy sparse matrix: {(end-stop):.03f} secs.")

Time with NumPy coefs matrix: 1.415 secs.
Time with SciPy sparse matrix: 0.703 secs.
